# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def nhl_correlation():
    # YOUR CODE HERE
    # raise NotImplementedError()
    nhlcities = cities.iloc[:,[0,1,-1]]
    nhlcities = nhlcities[nhlcities.NHL != '—']
    nhlcities = nhlcities.reset_index()
    nhlcities = nhlcities.iloc[:,[1,2,3]]
    nhlcities['NHL'] = nhlcities['NHL'].str.replace(r"\[.*\]","")
    nhlcities = nhlcities[nhlcities.NHL != '']
    nhlcities = nhlcities.reset_index()
    nhlcities = nhlcities.iloc[:,[1,2,3]]
    nhlcities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)

    def separate(x):
        mylist = re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z][a-z]+', x)
        return mylist

    nhlcities['NHL'] = nhlcities['NHL'].apply(lambda x: separate(x))
    teams = nhlcities['NHL']
    area = nhlcities['Metropolitan area'] 

    nhl_ = {}
    for idx,team in enumerate(nhlcities['NHL']) :
        for unit in team:
            nhl_[unit] = area[idx]
    newnhl = pd.DataFrame(nhl_.items(), columns=['team', 'Metropolitan area'])
    newnhl['team'] = newnhl['team'].str.replace(r'[A-Z][a-z]+ ','')
    premerge = pd.merge(newnhl,nhlcities,'outer', on = 'Metropolitan area')
    premerge = premerge.iloc[:,[0,1,2]]
    nhl = nhl_df.head(35)
    nhl = nhl[nhl['team'].str.contains('Division')==False]
    
    nhl = nhl.reset_index()
    nhl = nhl.iloc[:,[1,3,4]]
    nhl['team'] = nhl['team'].str.replace(r"\*","")
    nhl['team'] = nhl['team'].str.replace(r"[A-Z][a-z]+\.","")
    nhl['team'] = nhl['team'].str.replace(r"[A-Z][a-z]+ ","")
    nhl['team'] = nhl['team'].str.replace(r" ","")
    nhl = nhl.astype({'team': str,'W': int, 'L': int})
    nhl['percent'] = nhl['W']/(nhl['W']+nhl['L'])
    merge=pd.merge(premerge,nhl,'outer', on = 'team')
    merge.rename(columns = {'Metropolitan area':'Metro Area'}, inplace = True)
    merge = merge.astype({'percent': float,'Population': int})
    merge = merge.groupby('Metro Area').agg({'percent': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = merge['Population']
    win_loss_by_region = merge['percent'] 

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [4]:
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    
    nbacities = cities.iloc[:,[0,1,4]]
    nbacities = nbacities[nbacities.NBA != '—']
    nbacities = nbacities.reset_index()
    nbacities = nbacities.iloc[:,[1,2,3]]
    nbacities['NBA'] = nbacities['NBA'].str.replace(r"\[.*\]","")
    nbacities = nbacities[nbacities.NBA != '']
    nbacities = nbacities.reset_index()
    nbacities = nbacities.iloc[:,[1,2,3]]
    nbacities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
    def separate(x):
        if x[:1].isdigit():
            y = [x]
            return y
        mylist = re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z][a-z]+', x)
        return mylist
    nbacities['NBA'] = nbacities['NBA'].apply(lambda x: separate(x))
    teams = nbacities['NBA']
    area = nbacities['Metropolitan area'] 
    nba_ = {}
    for idx,team in enumerate(nbacities['NBA']) :
        for unit in team:
            nba_[unit] = area[idx]
    newnba = pd.DataFrame(nba_.items(), columns=['team', 'Metropolitan area'])
    newnba['team'] = newnba['team'].str.replace(r'[A-Z][a-z]+ ','')
    newnba['team'] = newnba['team'].str.replace(r' ','')
    premerge = pd.merge(newnba,nbacities,'outer', on = 'Metropolitan area')
    premerge = premerge.iloc[:,[0,1,2]]
    premerge = premerge.sort_values('team', ascending=True)
    premerge = premerge.reset_index()
    premerge = premerge.iloc[:,[1,2,3]]
    
    nba = nba_df.head(30)
    nba = nba.iloc[:,[0,1,2,3]]
    nba['team'] = nba['team'].str.replace(r"\*","")
    nba['team'] = nba['team'].str.replace(r"[A-Z][a-z]+ ","")
    nba['team'] = nba['team'].str.replace(r"\(([0-9])*\)","")
    nba['team'] = nba['team'].str.replace(r" ","")
    nba = nba.sort_values('team', ascending=True)
    nba.rename(columns = {'W/L%':'Percent'}, inplace = True)
    percent = nba.Percent
    # merge=pd.merge(premerge,nba,'outer', on = 'team')
    nba = nba.reset_index()
    percent = percent.reset_index()
    nba = nba.iloc[:,[1,2,3,4]]
    percent = percent.iloc[:,[1]]
    premerge['Percent'] = percent
    premerge.rename(columns = {'Metropolitan area':'Metro Area'}, inplace = True)
    premerge = premerge.astype({'Population': int,'Percent': float})
    premerge = premerge.groupby('Metro Area').agg({'Percent': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = premerge['Population']
    win_loss_by_region = premerge['Percent'] 
    
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    #raise NotImplementedError()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [6]:
nba_correlation()

-0.17636350642182938

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    
    mlbcities = cities.iloc[:,[0,1,3]]
    mlbcities = mlbcities[mlbcities.MLB != '—']
    mlbcities = mlbcities.reset_index()
    mlbcities = mlbcities.iloc[:,[1,2,3]]
    mlbcities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
    mlbcities['MLB'] = mlbcities['MLB'].str.replace(r"\[.*\]","")
    mlbcities = mlbcities[mlbcities.MLB != '']
    def separate(x):
        mylist = re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z][a-z]+', x)
        return mylist
    mlbcities['MLB'] = mlbcities['MLB'].apply(lambda x: separate(x))
    mlbcities = mlbcities.reset_index()
    mlbcities = mlbcities.iloc[:,[1,2,3]]
    teams = mlbcities['MLB']
    area = mlbcities['Metropolitan area'] 

    mlb_ = {}
    for idx,team in enumerate(mlbcities['MLB']) :
        for unit in team:
            mlb_[unit] = area[idx]
    newmlb = pd.DataFrame(mlb_.items(), columns=['team', 'Metropolitan area'])

    #newmlb['team'] = newmlb['team'].str.replace(r' ','')
    premerge = pd.merge(newmlb,mlbcities,'outer', on = 'Metropolitan area')
    premerge = premerge.iloc[:,[0,1,2]]
    premerge = premerge.sort_values('team', ascending=True)
    premerge = premerge.reset_index()
    premerge = premerge.iloc[:,[1,2,3]]
    
    mlb = mlb_df.head(30)
    mlb = mlb.iloc[:,[0,3]]
    def separate_(x):
        if x == 'Boston Red Sox':
            y = 'Red Sox'
            return y
        if x == 'Chicago White Sox':
            y = 'White Sox'
            return y
        if x == 'Toronto Blue Jays':
            y = 'Blue Jays'
            return y
        else:
            y = x.split(' ')[-1]
            return y
    mlb['team'] = mlb['team'].apply(lambda x: separate_(x))
    mlb = mlb.sort_values('team', ascending=True)
    mlb.rename(columns = {'W-L%':'Percent'}, inplace = True)
    mlb = mlb.reset_index()
    mlb = mlb.iloc[:,[1,2]]

    percent = mlb.Percent
    premerge['Percent'] = percent

    premerge.rename(columns = {'Metropolitan area':'Metro Area'}, inplace = True)
    premerge = premerge.astype({'Population': int,'Percent': float})
    premerge = premerge.groupby('Metro Area').agg({'Percent': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = premerge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = premerge['Percent'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    #raise NotImplementedError()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [8]:
mlb_correlation()

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE

    cities3=cities.iloc[:,[0,1,2]]
    cities3 = cities3[cities3.NFL != '—']
    cities3 = cities3.reset_index()
    cities3 = cities3.iloc[:,[1,2,3]]
    cities3.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
    cities3['NFL'] = cities3['NFL'].str.replace(r"\[.*\]","")
    cities3 = cities3[cities3.NFL != '']
    cities3 = cities3[cities3.NFL != '— ']
    def separatenfl(x):
        mylist = re.findall('[0-9]+[a-z]+|[A-Z][a-z]+', x)
        return mylist
    cities3['NFL'] = cities3['NFL'].apply(lambda x: separatenfl(x))
    cities3 = cities3.reset_index()
    cities3 = cities3.iloc[:,[1,2,3]]
    teams = cities3['NFL']
    area = cities3['Metropolitan area'] 
    nfl_ = {}
    for idx,team in enumerate(cities3['NFL']) :
        for unit in team:
            nfl_[unit] = area[idx]

    newnfl = pd.DataFrame(nfl_.items(), columns=['team', 'Metropolitan area'])
    premerge3 = pd.merge(newnfl,cities3,'outer', on = 'Metropolitan area')
    premerge3 = premerge3.iloc[:,[0,1,2]]
    premerge3 = premerge3.sort_values('team', ascending=True)
    premerge3 = premerge3.reset_index()
    premerge3 = premerge3.iloc[:,[1,2,3]]
    
    nfl = nfl_df[nfl_df['year'] == 2018]
    def idennfl(x):
        y = x
        mylist = re.findall('AFC|NFC', y)
        if mylist == []:
            return x
        else: 
            return '-'
    nfl['team'] = nfl['team'].apply(lambda x: idennfl(x))
    nfl = nfl[nfl.team != '-']
    nfl.rename(columns = {'W-L%':'Percent'}, inplace = True)
    nfl = nfl.reset_index()
    nfl = nfl.iloc[:,[-2,-3]]
    nfl['team'] = nfl['team'].str.replace(r"\*|\+","")

    nfl['team'] = nfl['team'].str.replace(r"\S+ ","")
    nfl = nfl.sort_values('team', ascending=True)
    nfl = nfl.reset_index()

    percent = nfl.Percent

    premerge3['Percent'] = percent
    premerge3.rename(columns = {'Metropolitan area':'Metro Area'}, inplace = True)
    premerge3 = premerge3.astype({'Population': int,'Percent': float})
    premerge3 = premerge3.groupby('Metro Area').agg({'Percent': np.nanmean, 'Population': np.nanmean})
    
    #raise NotImplementedError()
    
    population_by_region = premerge3['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = premerge3['Percent']  # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [17]:
nfl_correlation()

0.004282141436393017

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def separate(x):
        mylist = re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z][a-z]+', x)
        return mylist
def separate1(x):
        if x[:1].isdigit():
            y = [x]
            return y
        mylist = re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z][a-z]+', x)
        return mylist
def separate2(x):
        mylist = re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z][a-z]+', x)
        return mylist
def separate3(x):
        if x == 'Boston Red Sox':
            y = 'Red Sox'
            return y
        if x == 'Chicago White Sox':
            y = 'White Sox'
            return y
        if x == 'Toronto Blue Jays':
            y = 'Blue Jays'
            return y
        else:
            y = x.split(' ')[-1]
            return y
def separate4(x):
        mylist = re.findall('[0-9]+[a-z]+|[A-Z][a-z]+', x)
        return mylist
def iden(x):
        y = x
        mylist = re.findall('AFC|NFC', y)
        if mylist == []:
            return x
        else: 
            return '-'

def sports_team_performance():
    # YOUR CODE HERE
    
    nhlcities = cities.iloc[:,[0,1,-1]]
    nhlcities = nhlcities[nhlcities.NHL != '—']
    nhlcities = nhlcities.reset_index()
    nhlcities = nhlcities.iloc[:,[1,2,3]]
    nhlcities['NHL'] = nhlcities['NHL'].str.replace(r"\[.*\]","")
    nhlcities = nhlcities[nhlcities.NHL != '']
    nhlcities = nhlcities.reset_index()
    nhlcities = nhlcities.iloc[:,[1,2,3]]
    nhlcities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)

    

    nhlcities['NHL'] = nhlcities['NHL'].apply(lambda x: separate(x))
    teams = nhlcities['NHL']
    area = nhlcities['Metropolitan area'] 

    nhl_ = {}
    for idx,team in enumerate(nhlcities['NHL']) :
        for unit in team:
            nhl_[unit] = area[idx]
    newnhl = pd.DataFrame(nhl_.items(), columns=['team', 'Metropolitan area'])
    newnhl['team'] = newnhl['team'].str.replace(r'[A-Z][a-z]+ ','')
    premerge = pd.merge(newnhl,nhlcities,'outer', on = 'Metropolitan area')
    premerge = premerge.iloc[:,[0,1,2]]
    nhl = nhl_df.head(35)
    nhl = nhl[nhl['team'].str.contains('Division')==False]

    nhl = nhl.reset_index()
    nhl = nhl.iloc[:,[1,3,4]]
    nhl['team'] = nhl['team'].str.replace(r"\*","")
    nhl['team'] = nhl['team'].str.replace(r"[A-Z][a-z]+\.","")
    nhl['team'] = nhl['team'].str.replace(r"[A-Z][a-z]+ ","")
    nhl['team'] = nhl['team'].str.replace(r" ","")
    nhl = nhl.astype({'team': str,'W': int, 'L': int})
    nhl['percent'] = nhl['W']/(nhl['W']+nhl['L'])
    merge=pd.merge(premerge,nhl,'outer', on = 'team')
    merge.rename(columns = {'Metropolitan area':'Metro Area'}, inplace = True)
    merge = merge.astype({'percent': float,'Population': int})
    merge = merge.groupby('Metro Area').agg({'percent': np.nanmean, 'Population': np.nanmean})
    merge.rename(columns = {'percent':'Percent'}, inplace = True)
    
    cities1 = cities.iloc[:,[0,1,4]]
    cities1 = cities1[cities1.NBA != '—']
    cities1 = cities1.reset_index()
    cities1 = cities1.iloc[:,[1,2,3]]
    cities1['NBA'] = cities1['NBA'].str.replace(r"\[.*\]","")
    cities1 = cities1[cities1.NBA != '']
    cities1 = cities1.reset_index()
    cities1 = cities1.iloc[:,[1,2,3]]
    cities1.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
    
    cities1['NBA'] = cities1['NBA'].apply(lambda x: separate1(x))
    teams1 = cities1['NBA']
    area1 = cities1['Metropolitan area'] 
    nba_ = {}
    for idx,team in enumerate(cities1['NBA']) :
        for unit in team:
            nba_[unit] = area1[idx]
    newnba = pd.DataFrame(nba_.items(), columns=['team', 'Metropolitan area'])
    newnba['team'] = newnba['team'].str.replace(r'[A-Z][a-z]+ ','')
    newnba['team'] = newnba['team'].str.replace(r' ','')
    premerge1 = pd.merge(newnba,cities1,'outer', on = 'Metropolitan area')
    premerge1 = premerge1.iloc[:,[0,1,2]]
    premerge1 = premerge1.sort_values('team', ascending=True)
    premerge1 = premerge1.reset_index()
    premerge1 = premerge1.iloc[:,[1,2,3]]
    premerge1 = premerge1.head(30)
    nba = nba_df.head(30)
    nba = nba.iloc[:,[0,1,2,3]]
    nba['team'] = nba['team'].str.replace(r"\*","")
    nba['team'] = nba['team'].str.replace(r"[A-Z][a-z]+ ","")
    nba['team'] = nba['team'].str.replace(r"\(([0-9])*\)","")
    nba['team'] = nba['team'].str.replace(r" ","")
    nba = nba.sort_values('team', ascending=True)
    nba.rename(columns = {'W/L%':'Percent'}, inplace = True)
    percent = nba.Percent
    # merge=pd.merge(premerge,nba,'outer', on = 'team')
    nba = nba.reset_index()
    percent = percent.reset_index()
    nba = nba.iloc[:,[1,2,3,4]]
    percent = percent.iloc[:,[1]]
    premerge1['Percent'] = percent
    premerge1.rename(columns = {'Metropolitan area':'Metro Area'}, inplace = True)
    premerge1 = premerge1.astype({'Percent': float})
    premerge1 = premerge1.astype({'Population': int})
    #print(type(premerge1['Population'][0]))
    premerge1 = premerge1.groupby('Metro Area').agg({'Percent': np.nanmean, 'Population': np.nanmean})
    
    cities2 = cities.iloc[:,[0,1,3]]
    cities2 = cities2[cities2.MLB != '—']
    cities2 = cities2.reset_index()
    cities2 = cities2.iloc[:,[1,2,3]]
    cities2.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
    cities2['MLB'] = cities2['MLB'].str.replace(r"\[.*\]","")
    cities2 = cities2[cities2.MLB != '']
    
    cities2['MLB'] = cities2['MLB'].apply(lambda x: separate(x))
    cities2 = cities2.reset_index()
    cities2 = cities2.iloc[:,[1,2,3]]
    teams2 = cities2['MLB']
    area2 = cities2['Metropolitan area'] 

    mlb_ = {}
    for idx,team in enumerate(cities2['MLB']) :
        for unit in team:
            mlb_[unit] = area2[idx]
    newmlb = pd.DataFrame(mlb_.items(), columns=['team', 'Metropolitan area'])

    #newmlb['team'] = newmlb['team'].str.replace(r' ','')
    premerge2 = pd.merge(newmlb,cities2,'outer', on = 'Metropolitan area')
    premerge2 = premerge2.iloc[:,[0,1,2]]
    premerge2 = premerge2.sort_values('team', ascending=True)
    premerge2 = premerge2.reset_index()
    premerge2 = premerge2.iloc[:,[1,2,3]]

    mlb = mlb_df.head(30)
    mlb = mlb.iloc[:,[0,3]]
    
    mlb['team'] = mlb['team'].apply(lambda x: separate3(x))
    mlb = mlb.sort_values('team', ascending=True)
    mlb.rename(columns = {'W-L%':'Percent'}, inplace = True)
    mlb = mlb.reset_index()
    mlb = mlb.iloc[:,[1,2]]

    percent2 = mlb.Percent
    premerge2['Percent'] = percent

    premerge2.rename(columns = {'Metropolitan area':'Metro Area'}, inplace = True)
    premerge2 = premerge2.astype({'Population': int,'Percent': float})
    premerge2 = premerge2.groupby('Metro Area').agg({'Percent': np.nanmean, 'Population': np.nanmean})

    # NFL
    cities3=cities.iloc[:,[0,1,2]]
    cities3 = cities3[cities3.NFL != '—']
    cities3 = cities3.reset_index()
    cities3 = cities3.iloc[:,[1,2,3]]
    cities3.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
    cities3['NFL'] = cities3['NFL'].str.replace(r"\[.*\]","")
    cities3 = cities3[cities3.NFL != '']
    cities3 = cities3[cities3.NFL != '— ']
    def separatenfl(x):
        mylist = re.findall('[0-9]+[a-z]+|[A-Z][a-z]+', x)
        return mylist
    cities3['NFL'] = cities3['NFL'].apply(lambda x: separatenfl(x))
    cities3 = cities3.reset_index()
    cities3 = cities3.iloc[:,[1,2,3]]
    teams = cities3['NFL']
    area = cities3['Metropolitan area'] 
    nfl_ = {}
    for idx,team in enumerate(cities3['NFL']) :
        for unit in team:
            nfl_[unit] = area[idx]

    newnfl = pd.DataFrame(nfl_.items(), columns=['team', 'Metropolitan area'])
    premerge3 = pd.merge(newnfl,cities3,'outer', on = 'Metropolitan area')
    premerge3 = premerge3.iloc[:,[0,1,2]]
    premerge3 = premerge3.sort_values('team', ascending=True)
    premerge3 = premerge3.reset_index()
    premerge3 = premerge3.iloc[:,[1,2,3]]
    
    nfl = nfl_df[nfl_df['year'] == 2018]
    def idennfl(x):
        y = x
        mylist = re.findall('AFC|NFC', y)
        if mylist == []:
            return x
        else: 
            return '-'
    nfl['team'] = nfl['team'].apply(lambda x: idennfl(x))
    nfl = nfl[nfl.team != '-']
    nfl.rename(columns = {'W-L%':'Percent'}, inplace = True)
    nfl = nfl.reset_index()
    nfl = nfl.iloc[:,[-2,-3]]
    nfl['team'] = nfl['team'].str.replace(r"\*|\+","")

    nfl['team'] = nfl['team'].str.replace(r"\S+ ","")
    nfl = nfl.sort_values('team', ascending=True)
    nfl = nfl.reset_index()

    percent = nfl.Percent

    premerge3['Percent'] = percent
    premerge3.rename(columns = {'Metropolitan area':'Metro Area'}, inplace = True)
    premerge3 = premerge3.astype({'Population': int,'Percent': float})
    premerge3 = premerge3.groupby('Metro Area').agg({'Percent': np.nanmean, 'Population': np.nanmean})
    
    nhldf1 = merge
    nbadf1 = premerge1
    mlbdf1 = premerge2
    nfldf1 = premerge3
    list1 = [nfldf1, nbadf1, nhldf1, mlbdf1]
    tempmerge1 = pd.merge(nhldf1,nbadf1,'inner', on = 'Metro Area')
    
    data_set = {'NFL': nfldf1,
                'NBA': nbadf1,
                'NHL': nhldf1,
                'MLB': mlbdf1}
    
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for index in sports:
        for idx in sports:
            if index != idx:
                tempmerge = pd.merge(data_set[index],data_set[idx],'inner', on = 'Metro Area')
                p_values.loc[index, idx]=stats.ttest_rel(tempmerge['Percent_x'],tempmerge['Percent_y'])[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [19]:
sports_team_performance()

AssertionError: The MLB-NFL p-value should be around 0.80

In [20]:
p_values

NameError: name 'p_values' is not defined